# Lectura interactiva del Módulo 8

A lo largo de este curso estaremos trabajando con estas *Jupyter Notebooks*, tratando de poner en práctica los conceptos aprendidos en cada módulo. Pero, también las utilizaremos como herramienta de lectura para, antes de programar nuestro propio código, conocer cómo se usan algunas funciones, aprender sobre distintas librerías, etc.

En esta lectura generaremos modelos de regresión logística para resolver un problema de clasificación, a la vez que evaluaremos dichos modelos calculando su exactitud y generando una matriz de confusión.

Por favor no modifiques las celdas con las instrucciones, y solamente escribe código en las celdas donde así se te indica. **Si en algún momento seleccionas por error una celda de instrucciones y su apariencia cambia, simplemente presiona "Ctrl + Enter".**

Dentro de las celdas de código, las líneas que inician con un "\#" son comentarios y no se ejecutarán, simplemente sirven como instrucciones o descripciones útiles para ustedes.

Una vez más trabajaremos con la base de datos del *Dataset for estimation of obesity levels based on eating habitss and physical condition in individuals from Colombia, Peru and Mexico*, una base de datos que contiene 2,111 observaciones y 10 variables:
 - `Sexo`. Female para mujeres, Male para hombres.
 - `Edad`. En años.
 - `Estatura`. En metros.
 - `Peso`. En kilogramos.
 - `FamiliarConSobrepeso`. Si cuenta (Yes) o no (No) con un familiar con sobrepeso.
 - `ComeMuchasCalorías`. Si come (Yes) o no (No) comida con alto contenido calórico de forma frecuente.
 - `ComeVegetales`. Nunca (1), algunas veces (2) o nunca (3) come vegetales con sus comidas.
 - `Fumador`. Si fuma (Yes) o no (No).
 - `ConsumoDeAgua`. Cantidad de agua que consume diariamente: 1 si es menos de 1 litro, 2 si es entre 1 y 2 litros, 3 si es más de 2 litros.
 - `NivelDeObesidad`. Nivel de obesidad en una escala que incluye 7 diferentes niveles.
 
Los datos fueron proporcionados por Fabio Mendoza Palechor de la Universidad de la Costa, se descargaron directamente del [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Estimation+of+obesity+levels+based+on+eating+habits+and+physical+condition+), y se puede leer más información sobre los mismos en su [publicación](https://www.sciencedirect.com/science/article/pii/S2352340919306985) para la revista científica Data in brief.
 
En el módulo 6 usamos esta base de datos para predecir el peso de una persona a partir de cierta información. En esta ocasión, lo que buscaremos será clasificar a las personas, en una primera instancia, como persona con sobrepeso o persona sin sobrepeso. Posteriormente, las clasificaremos usando el sistema usado en el estudio de donde se obtuvieron los datos: peso insuficiente, peso normal, sobrepeso tipo I, sobrepeso tipo II, obesidad tipo I, obesidad tipo II, y obesidad tipo III.

Empecemos este ejercicio, como ya es costumbre, importando los datos al ambiente de trabajo. Partiendo de nuestro interés actual, sería sensato pensar que el peso de la persona será el primer indicador de su nivel de peso, por lo que también generaremos una gráfica de dispersión para visualizar el peso de las personas, estratificado por su nivel. Adicionalmente, escribiremos una línea de código que indique que las etiquetas de la gráfica en el eje "x" deben imprimirse de forma vertical (para mejorar la visualización), usando la función `xticks` y definiendo el parámetro `rotation="vertical"`. Es importante agregar a esta línea un `;` al final, para evitar un despliegue de información innecesaria.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    import pandas as pd<br>
    df = pd.read_csv("Obesidad.csv")<br>
    import matplotlib.pyplot as plt<br>
    %matplotlib inline<br>
    plt.scatter(df.NivelDeObesidad,df.Peso)<br>
    plt.xticks(rotation="vertical");
</details>

In [ ]:
# Importa la librería pandas

# Lee el archivo

# Importa matplotlib

# Función mágica para graficar en Jupyter Notebook

# Gráfica de dispersión

# Rotar etiquetas y evitar despliegue de información innecesaria


La gráfica nos muestra que, aunque claramente hay una relación entre el peso y el nivel de obesidad, hay mucho traslape de valores entre clases, por lo que la tarea no será tan sencilla. Para el primer análisis, nos interesa predecir su una persona tiene o no sobrepeso. Por lo mismo, si una persona pertenece a las clases `Normal_Weight` o `Insufficient_Weight` será considerada como sujeto sin sobrepeso, en caso contrario, como sujeto con sobrepeso.

Hay múltiples formas de realizar esta tarea. En la siguiente celda, primero generaremos variables *dummy* para la variable `NivelDeObesidad`. Después, generaremos una variable de nombre `SinSobrepeso` donde se identifique con un 1 a los sujetos sin sobrepeso y con un 0 sujetos con sobrepeso. Esto se logra de forma sencilla con las variables *dummy* y haciendo uso del operador lógico OR, denotado por `|` en Python. Posteriormente, generaremos una variable de nombre `ConSobrepeso` donde se almacene la información contraria: 1 para observaciones con sobrepeso, 0 para observaciones sin sobrepeso. Esto se logra realizando la operación `1 - SinSobrepeso`. Adicionalmente, almacenaremos esta variable como un data frame de pandas, definiendo el nombre de la misma usando la estructura `pd.DataFrame({"nombre": contenido})`.

Después, concatenaremos esta información al data frame original con la función `concat`. Finalmente, imprimiremos un resumen de los datos para verificar que se cuente con las columnas adecuadas, así como un conteo de los valores de la nueva variable.

**Nota:** Es posible que al correr la misma celda múltiples veces se genere un error, si eso te llega a suceder, simplemente vuelve a ejecutar las celdas, desde la primera y hasta el punto en que te encuentras.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    dummyPeso = pd.get_dummies(df.NivelDeObesidad)<br>
    SinSobrepeso = dummyPeso.Normal_Weight | dummyPeso.Insufficient_Weight<br>
    ConSobrepeso = pd.DataFrame({"ConSobrepeso":1 - SinSobrepeso})<br>
    df = pd.concat([df,ConSobrepeso],axis=1)<br>
    print(df.head())<br>
    print(df.ConSobrepeso.value_counts())
</details>

In [ ]:
# Crear variables dummy

# Crear variable SinSobrepeso

# Crear data frame ConSobrepeso

# Concatenar información al data frame original

# Imprimir resumen

# Imprimir conteo de valores


Todo se ve bien, nuestro data frame ahora tiene una variable de nombre `ConSobrepeso` que indica un 0 para las clases `Normal_Weight` e `Insufficient_Weight` y un 1 para el resto de las clases. El conteo de valores muestra que contamos con 1,552 sujetos con sobrepeso y 559 sujetos sin sobrepeso. Este desbalance de clases puede llegar a ser problemático cuando se entrene al modelo, y deberemos tenerlo en cuenta más adelante.

Pero, ¿por qué podría ser problemático? Imaginemos una situación extrema, una base de datos de mil mujeres que se realizaron una mamografía de pecho de rutina, dentro de las cuales solamente a 2 de ellas se les detectó cáncer de pecho. Esta situación, aunque extrema en términos de desbalance de clases, no es extraña ni imposible, pues estamos hablando de revisiones de rutina, no de revisiones realizadas por alguna sospecha previa.

Si entrenamos un modelo de inteligencia artificial para clasificar a las pacientes dependiendo de si tienen o no cáncer, y no tenemos el cuidado suficinte para considerar el desbalance de clases en la etapa de entrenamiento, será muy probable que terminemos con un modelo que jamás prediga que una nueva observación pertenece a una mujer con cáncer de pecho. ¿Por qué? Porque en nuestros datos de entrenamiento, si predice un 0 (no cáncer) para todas las observaciones, terminará con una exactitud de 99.8%, pues 998 de las mil pacientes no tienen cáncer.

Existen muchas metodologías distintas para tratar de corregir este problema, aumentar la base de datos para tratar de balancear las clases, considerar tamaños similares en la etapa de entrenamiento, modificar la métrica utilizada para considerar el efecto de la cantidad de sujetos por clase, etc. Más adelante verán que podemos implementar dicha solución de forma extremadamente sencilla.

De momento, sigamos avanzando con nuestro análisis y dividamos nuestros datos en entrenamiento y prueba, definiendo una proporción de 70/30. Pero, por el momento, solamente usaremos el peso como nuestra variable de entrada. Revisemos si el desbalance se mantiene en los datos de entrenamiento y en los de validación, imprimiendo el conteo de valores de ambos subconjuntos de datos.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    from sklearn.model_selection import train_test_split<br>
    X_train, X_test, Y_train, Y_test = train_test_split(df.Peso,df.ConSobrepeso,train_size = .7)<br>
    print(Y_train.value_counts())<br>
    print(Y_test.value_counts())
</details>

In [ ]:
# Importar función

# Dividir datos

# Imprimir conteo en entrenamiento

# Imprimir conteo en prueba


Como era de esperarse, se sigue observando un desbalance de clases: 1,070 observaciones con sobrepeso contra 407 observaciones sin sobrepeso en los datos de entrenamiento (los números pueden variar dependiendo de la división de datos). Es momento de crear y entrenar al modelo de regresión logística. Utilizaremos la función `LogisticRegression` de la librería sklearn.linear_model; definiremos `penalty="none"`, pues de momento no queremos realizar un proceso de penalización que ayude a eliminar variables (por ejemplo, Lasso, como lo vimos en el módulo 6).

Para tratar de minimizar el efecto causado por el desbalance de clases, también definiremos `class_weight="balanced"`, de forma que se agregue un peso a cada clase cuando se estime la función de verisimilitud. A una clase con muy pocas muestras se le dará un peso grande, y a una clase con muchas muestras se le dará un peso pequeño. De esta forma se trata de que la precisión en la predicción de ambas clases terminen teniendo la misma importancia para el modelo.

El entrenamiento del modelo lo realizaremos usando los datos de entrenamiento generados en el paso anterior. Como solamente estamos usando una variable de entrada, necesitamos cambiar la forma de la variable, de modo que sea aceptado por la función `fit`. Lograremos esto con el término `array.shape(-1,1)` que cambiará nuestros datos de tene una forma (1477,) a tener una forma (1477,1).<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    from sklearn.linear_model import LogisticRegression<br>
    model = LogisticRegression(penalty="none",class_weight="balanced").fit(X_train.array.reshape(-1,1),Y_train)
</details>

In [ ]:
# Importar función

# Generar y entrenar modelo


Es momento de evaluar a nuestro modelo. Primero revisaremos la exactitud, calculada como la cantidad de predicciones correctas dividido entre la cantidad de predicciones realizadas. Para generar las predicciones, usaremos la función `predict`, especificando como parámetro los datos de prueba, modificando su forma de la misma forma que en el paso anterior.

Despues, compararemos el vector de predicciones con el vector de clases reales (datos de validación). Si la clase predicha es igual a la clase real, se considera como un acierto. Este proceso entregará como resultado un vector de 0s (fallos) y 1s (aciertos). Si calculamos el promedio de dicho vector (suma de aciertos entre cantidad de predicciones totales), habremos calculado la exactitud del modelo. Imprimamos el resultado en consola para evaluar el desempeño del modelo.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    import numpy as np<br>
    pred = model.predict(X_test.array.reshape(-1,1))<br>
    acc = np.mean(pred == Y_test)<br>
    print(acc)
</details>

In [ ]:
# Importar numpy

# Generar predicciones

# Calcular exactitud

# Imprimir valor


El modelo tiene un buen desempeño, aproximadamente predice de forma correcta al 90% de las observaciones (variaciones dependientes de la división de datos). Si nos interesara investigar un poco más a fondo, podríamos revisar la exactitud por clase, conocida como sensibilidad (predicciones correctas en clase 1 sobre cantidad de observaciones de clase 1) y especificidad (predicciones correctas en clase 0 sobre cantidad de observaciones de clase 0. Dejaremos esa tarea para más tarde.

Ahora, convirtamos nuestro modelo de regresión logística simple, en un modelo de regresión logística múltiple. Primero generaremos variables dummy para las columnas `Sexo`, `FamiliarConSobrepeso`, `ComeMuchasCalorias`, y `Fumador` usando la función `get_dummies()`, igual que como hicimos antes, pero nos aseguraremos de incluir el término `prefix="Nombre"` para asignar un nombre adecuado a cada variable generada, así como el término `drop_first=True` para evitar variables redundantes. Después, concatenaremos estas variables en un nuevo data frame. A su vez, eliminaremos de la base de datos justamente estas columnas, las que nos ayudaron a generar las vaiables dummy. Pero, aprovecharemos esa acción para también eliminar a las variables `ComeVegetales` y `ConsumoDeAgua`, que en una lectura previa demostramos que tenía información no confiable, así como la variable `NivelDeObesidad`, que de momento solo se utilizó para generar la variable de respuesta `ConSobrepeso`. Al final imprimiremos un resumen del nuevo data frame para validar su estructura.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    dSexo = pd.get_dummies(df.Sexo,drop_first=True)<br>
    dFamiliar = pd.get_dummies(df.FamiliarConSobrepeso,drop_first=True,prefix="FamiliarConSobrepeso")<br>
    dCome = pd.get_dummies(df.ComeMuchasCalorias,drop_first=True,prefix="ComeMuchasCalorias")<br>
    dFumador = pd.get_dummies(df.Fumador,drop_first=True,prefix="Fumador")<br>
    df2 = pd.concat([df,dSexo,dFamiliar,dCome,dFumador],axis=1)<br>
    df2=df2.drop(["Sexo","FamiliarConSobrepeso","ComeMuchasCalorias","Fumador","ComeVegetales","ConsumoDeAgua","NivelDeObesidad"],axis=1)<br>
    print(df2.head())
</details>

In [ ]:
# Generar variables dummy




# Concatenar data frame original y nuevas variables en un nuevo data frame

# Eliminar variables irrelevantes

# Imprimir resumen


Perfecto, ahora nuestra base de datos solamente contiene variables de interés, ya sean numéricas o categóricas procesadas para manipularse como numéricas. En la siguiente celda dividiremos los datos en entrenamiento y prueba, tal y como lo hicimos previamente, aunque ahora no necesitamos realizar ninguna transformación a los datos. También generaremos y entrenaremos al modelo usando los nuevos subconjuntos de datos, y finalmente calcularemos la exactitud del modelo.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    X2_train, X2_test, Y2_train, Y2_test = train_test_split(df2.drop("ConSobrepeso",axis=1),df2.ConSobrepeso,train_size = .7)<br>
    model2 = LogisticRegression(penalty='none',class_weight="balanced").fit(X2_train,Y2_train)<br>
    pred2 = model2.predict(X2_test)<br>
    acc2 = np.mean(pred2 == Y2_test)<br>
    print(acc2)
</details>

In [ ]:
# Dividir datos

# Generar y entrenar modelo

# Generar predicciones

# Calcular exactitud

# Imprimir valor


Excelente, una exactitud de aproximadamente 98% (variación dependiente de división de datos). Sin embargo, solamente estamos clasificando entre personas con sobrepeso y personas sin sobrepeso. Tal vez, una tarea más compleja sea clasificar entre persona con sobrepeso tipo I y persona con sobrepeso tipo II, o persona con obesidad tipo II y persona con obesidad tipo III.

Realicemos entonces un nuevo análisis, en el que llevemos a cabo una regresión logística multinomial múltiple. Empecemos generando un data frame que nos sirva para esta tarea. La idea es muy similar a la construcción del data frame anterior, solo cambiando la variable de respuesta; en vez de incluir la columna `ConSobrepeso` y eliminar `NivelDeObesidad`, haremos lo contrario. Al finalizar, como de costumbre, imprimiremos un resumen del nuevo data frame para revisar que todo esté en orden.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    df3 = pd.concat([df,dSexo,dFamiliar,dCome,dFumador],axis=1)<br>
    df3 = df3.drop(["Sexo","FamiliarConSobrepeso","ComeMuchasCalorias","Fumador","ComeVegetales","ConsumoDeAgua","ConSobrepeso"],axis=1)<br>
    print(df3.head())
</details>

In [ ]:
# Concatenar en un nuevo data frame, df con las variables dummy

# Eliminar variables que no interesan

# Imprimir resumen


Todo se ve bien, contamos con las variables de interés para este análisis. Dividamos los datos en entrenamiento y prueba con la misma metodología previa, y generemos y entrenemos el modelo. Para la división de datos necesitamos especificar que en X deben aparecer todas las variables, excepto `NivelDeObesidad`, mientras que Y debe conformarse solamente por dicha variable. Para la creación del modelo usaremos la misma función utilizada previamente, pero agregaremos el término `multi_class="multinomial"`, que indica que trabajaremos justamente con un modelo multinomial. Para entrenar el modelo usaremos la función `fit` con los datos de entrenamiento.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    X3_train, X3_test, Y3_train, Y3_test = train_test_split(df3.drop("NivelDeObesidad",axis=1),df3.NivelDeObesidad,train_size = .7)<br>
    model3 = LogisticRegression(penalty='none',class_weight="balanced",multi_class="multinomial").fit(X3_train,Y3_train)
</details>

In [ ]:
# Dividir datos

# Crear y entrenar modelo


La generación y entrenamiento de modelo probablemente generó algo de información; de momento no nos preocupemos por ella. Ahora revisemos la exactitud del modelo multinomial. Es de esperarse un desempeño más bajo que los modelos enfocados en simplemente clasificar a los sujetos como con sobrepeso o sin sobrepeso, pues ahora la tarea es mucho más complicada.

Utilizaremos una vez más la función `predict` para obtener las predicciones del modelo, que indica directamente la clase predicha, dependiendo de las probabilidades calculadas internamente. En la siguiente celda, genera dichas predicciones e imprímelas en consola.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    pred3 = model3.predict(X3_test)<br>
    print(pred3)
</details>

In [ ]:
# Genera la predición

# Imprime las predicciones


Justo como lo había comentado, la función `predict` indica para cada observación, la clase con mayor probabilidad dentro del modelo. Ahora revisemos la exactitud del modelo, de la misma forma que lo hicimos previamente, comparando las clases que el modelo predice contra las clases reales, y sacando el promedio de dicho vector.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    acc3 = np.mean(pred3 == Y3_test)<br>
    print(acc3)
</details>

In [ ]:
# Calcula exactitud

# Imprime el valor


Como lo esperábamos, la exactitud es más baja, aunque el modelo sigue siendo bueno. En los análisis multinomiales es común estar interesados en otras métricas, como la sensibilidad y especificidad mencionados previamente. Una forma sencilla de, con una sola imagen, mostrar mucha más información, son las matrices de confusión. Generemos una y después tratemos de interpretar su información.

Para generar la información de la matriz usaremos la función `confusion_matrix` y para graficarla, la función `ConfusionMatrizDisplay`, ambas de la librería sklearn.metrics. La función que genera la información de la matriz simplemente necesita el vector de clases eal y el vector de predicciones, y la función que se usa para visualización requiere precisamente de dicha matriz. Adicionalmente, le especificaremos los nombres de las clases con el término `display_labels=model3.classes`, y le agregaremos la función `plot` con el término `xticks_rotatio="vertical"` para que el texto sea más legible.<br><br>

<details>
    <summary>Si tienes problemas, da un click aquí para mostrar la solución</summary>
    from sklearn.metrics import confusion_matrix<br>
    from sklearn.metrics import ConfusionMatrixDisplay as cmd<br>
    cm = confusion_matrix(Y3_test,pred3)<br>
    cmd(cm,display_labels=model3.classes_).plot(xticks_rotation="vertical")
</details>

In [ ]:
# Importar funciones


# Generar matriz de confusión

# Visualizar matriz de confusión


Tratemos de analizar la imagen para poder interpretarla. En el eje "y" se tienen las verdaderas clases, mientras que en el eje "x" las predicciones realizadas. La diagonal principal entonces indica la cantidad de observaciones con predicciones correctas, y cualquier valor fuera de dicha diagonal indica un error del modelo.

En mi gráfica (en la tuya habrá variaciones por la división de datos) se muestra un 26 en la posición que une `Normal_weight` como clase real e `Insufficient_Weight` como la clase predecida. Eso indica que hubo 26 sujetos con un peso normal, que el modelo clasificó como sujetos con peso insuficiente.

Por lo mismo, esta visualización nos ayuda a no solamente saber qué tan exacto o preciso es el modelo, sino también a comprender en qué situaciones está cometiendo errores y en qué situaciones funciona muy bien. Por ejemplo, en mi caso, a todos los sujetos de la clase `Obesity_Type_II` y `Obesity_Type_III` los clasificó correctamente, mientras que de los 80 sujetos con peso normal, solo a 40 de hechos los clasificó con dicha clase, al resto los clasificó con peso normal o con uno de los dos tipos de sobrepeso.

Con esto terminamos la lectura interactiva del último moódulo de este curso, espero que las hayas disfrutado y que hayas generado nuevos conocimientos al utilizarlas.